# Deep Learning on Cassava Leaves Diseases 

This cassava dataset was shared on Kaggle https://www.kaggle.com/datasets/visalakshiiyer/cassava-image-dataset, disease classification.

In [19]:
# Required Libraries
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate
from tensorflow.keras.layers import LeakyReLU, Dropout, BatchNormalization
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint
import datetime 
import random
import pandas as pd

In [18]:
# Initialize random seed for reproducibility
seed = 42
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)

# Constants
IMG_SIZE = 244
NUM_CLASSES = 3 # Leaf conditions
NUM_CLASSES_SEG = 1 # 0: background, 1: foreground
BATCH_SIZE = 32
INITIAL_LEARNING_RATE = 0.001
NUM_EPOCHS = 50

## U-Net Definition

To identify the leaf in the image as the region of interest (ROI), I will adapt a custom U-Net model (primarily used in medical applications) to segment the leaf from its background. 

In [3]:
# Data Augmentation parameters
data_gen_args = dict(
    rotation_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.05,
    zoom_range=0.05,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2],
    fill_mode='nearest'
)

# Function to load images from a specified folder
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        if filename.endswith('.jpg'):
            img = cv2.imread(os.path.join(folder, filename))
            if img is not None:
                images.append(img)
    return images

# Function to create model
def create_model(img_shape, num_classes):
    inputs = Input(img_shape)

    # Encoder (downsampling)
    conv1 = Conv2D(64, (3, 3), padding='same')(inputs)
    conv1 = LeakyReLU()(conv1)
    conv1 = Dropout(0.1)(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = BatchNormalization()(conv2)
    conv2 = Dropout(0.1)(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    # Bridge
    bridge = Conv2D(256, (3, 3), activation='relu', padding='same')(pool2)
    bridge = BatchNormalization()(bridge)
    bridge = Dropout(0.1)(bridge)

    # Decoder (upsampling)
    up3 = concatenate([UpSampling2D(size=(2, 2))(bridge), conv2], axis=-1)
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(up3)
    conv3 = BatchNormalization()(conv3)
    conv3 = Dropout(0.1)(conv3)

    up4 = concatenate([UpSampling2D(size=(2, 2))(conv3), conv1], axis=-1)
    conv4 = Conv2D(64, (3, 3), activation='relu', padding='same')(up4)
    conv4 = BatchNormalization()(conv4)
    conv4 = Dropout(0.1)(conv4)

    # Output layer
    output = Conv2D(num_classes, (1, 1), activation='sigmoid')(conv4)

    model = Model(inputs=inputs, outputs=output)

    return model

# Function to calculate Dice Coefficient
def dice_coef(y_true, y_pred):
    smooth = 1.0
    y_true_f = tf.reshape(y_true, [-1])
    y_pred_f = tf.reshape(y_pred, [-1])
    intersection = tf.reduce_sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true_f) + tf.reduce_sum(y_pred_f) + smooth)

def dice_coef_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred)

def bce_dice_loss(y_true, y_pred):
    return tf.keras.losses.binary_crossentropy(y_true, y_pred) + dice_coef_loss(y_true, y_pred)

# Function to create mask for plant leaves
def create_mask_for_plant(image):
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    lower_hue = np.array([25, 50, 50])
    upper_hue = np.array([100, 255, 255])
    mask = cv2.inRange(hsv, lower_hue, upper_hue)
    return mask

def segment_plant(image):
    mask = create_mask_for_plant(image)
    output = cv2.bitwise_and(image, image, mask = mask)
    return output

def sharpen_image(image):
    # Using high-pass filter for image sharpening
    kernel = np.array([[0, -1, 0], [-1, 5,-1], [0, -1, 0]])
    image_sharp = cv2.filter2D(image, -1, kernel)
    return image_sharp


Leaf Masking

In [ ]:
# Load some sample images and visualize
sample_folder = './Cassava/brown_streak/'
sample_images = load_images_from_folder(sample_folder)

# Visualize a sample image
plt.imshow(cv2.cvtColor(sample_images[0], cv2.COLOR_BGR2RGB))
plt.title("Sample Original Image")
plt.show()

# Visualize a sample mask
sample_mask = create_mask_for_plant(sample_images[0])
plt.imshow(sample_mask, cmap='viridis')
plt.title("Sample Mask")
plt.show()


In [ ]:
# Required Libraries
import matplotlib.pyplot as plt
import seaborn as sns

# Disable warnings
import warnings
warnings.filterwarnings("ignore")

# Load images from each folder
brown_streak = load_images_from_folder('./Cassava/brown_streak/')
healthy = load_images_from_folder('./Cassava/healthy1')
mosaic_disease = load_images_from_folder('./Cassava/mosaic_disease1')

# Concatenate all the images
images = brown_streak + healthy + mosaic_disease

# Data for plotting
data_labels = ['brown_streak']*len(brown_streak) + ['healthy']*len(healthy) + ['mosaic_disease']*len(mosaic_disease)

# Create the countplot with the viridis palette
sns.countplot(x=data_labels, palette="viridis")

# Add a title and labels for better interpretation
plt.title('Distribution of Labels in the Dataset', fontsize=16)
plt.xlabel('Leaf Condition', fontsize=14)
plt.ylabel('Count', fontsize=14)

# Display value counts above bars for clarity
for p in plt.gca().patches:
    plt.gca().annotate(f'{p.get_height()}', (p.get_x() + p.get_width() / 2., p.get_height()),
                        ha='center', va='baseline', fontsize=12)

# Show the plot
plt.show()

# Resize the images and normalize
resized_images = [cv2.resize(image, (IMG_SIZE, IMG_SIZE)) for image in images]

# Segment images and create labels
labels = []
images_segmented = []
for image in resized_images:
    image_segmented = segment_plant(image)
    image_sharpen = sharpen_image(image_segmented)
    images_segmented.append(image_sharpen)

    image_gray = cv2.cvtColor(image_sharpen, cv2.COLOR_BGR2GRAY)
    _, label = cv2.threshold(image_gray, 1, 1, cv2.THRESH_BINARY)
    labels.append(label)

# Convert to numpy arrays and adjust types
images_segmented = np.array(images_segmented)
labels = np.array(labels).astype('float32')

# Create a combined labels array
combined_labels = [0]*len(brown_streak) + [1]*len(healthy) + [2]*len(mosaic_disease)

# Stratified shuffle split for train-val sets
stratSplit = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=seed)

for train_index, val_index in stratSplit.split(images_segmented, combined_labels):
    x_train, x_val = images_segmented[train_index], images_segmented[val_index]
    x_val_original = np.array(resized_images)[val_index] # For visualization 
    y_train, y_val = labels[train_index], labels[val_index]

# Data Augmentation using ImageDataGenerator
image_datagen = ImageDataGenerator(**data_gen_args)
mask_datagen = ImageDataGenerator(**data_gen_args)

# Expand y_train dimensions
y_train = np.expand_dims(y_train, -1)

# Fit the data generators
image_datagen.fit(x_train, augment=True, seed=seed)
mask_datagen.fit(y_train, augment=True, seed=seed)

# Create the generators
image_generator = image_datagen.flow(x_train, batch_size=BATCH_SIZE, seed=seed)
mask_generator = mask_datagen.flow(y_train, batch_size=BATCH_SIZE, seed=seed)

# Combine generators
train_generator = zip(image_generator, mask_generator)

# Create U-Net model
unet_model = create_model((IMG_SIZE, IMG_SIZE, 3), NUM_CLASSES_SEG)

# Print the model summary
unet_model.summary()

# Data Sampling

In [ ]:
# Required Libraries
import matplotlib.pyplot as plt
import random

# Define categories and their corresponding images
categories = ['brown_streak', 'healthy', 'mosaic_disease']
category_images = [brown_streak, healthy, mosaic_disease]

# Create subplots for each category
plt.figure(figsize=(10, 5))

# Iterate through categories and select one random image per category
for i, category in enumerate(categories):
    random_image = random.choice(category_images[i])
    
    # Plot the randomly selected image in the corresponding subplot
    plt.subplot(1, 3, i + 1)
    plt.imshow(cv2.cvtColor(random_image, cv2.COLOR_BGR2RGB))  # Convert BGR to RGB for correct display
    plt.title(f'Sampled Image ({category})', fontsize=12)
    plt.axis('off')

plt.tight_layout()
plt.show()


# Training and Evaluation of U-Net Model

In [5]:
# Required Libraries
import datetime
import tensorflow as tf
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

# ---------------- Initialization Section ----------------

# Debug print model initialization has started
print("\n=== Model Initialization ===\n")

# Setting Callbacks
# Logging directory for TensorBoard
log_dir = "logs/fit/unet model" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

# Initialize TensorBoard, EarlyStopping, ModelCheckpoint, and ReduceLROnPlateau callbacks
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)
early_stopping = EarlyStopping(monitor='val_dice_coef', mode='max', verbose=1, patience=5, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('unet_best_model.keras', monitor='val_dice_coef', mode='max',
                                    verbose=1, save_weights_only=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.001)  # Adjust learning rate

# Setting Learning Rate Schedule
# Using Exponential Decay for the learning rate
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    INITIAL_LEARNING_RATE, decay_steps=100000, decay_rate=0.96, staircase=True
)

# ---------------- Compilation Section ----------------

# Debug print model compilation has started
print("\n=== Model Compilation ===\n")

# Compile the U-Net Model
# Metrics include Dice coefficient, accuracy, precision, and recall
unet_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
    loss=bce_dice_loss,
    metrics=[dice_coef, 'accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()],
)

# ---------------- Training Section ----------------

# Debug print model training has started
print("\n=== Model Training ===\n")

# Train the U-Net Model
# Using the data generator for training and validation datasets
history = unet_model.fit(
    train_generator,
    steps_per_epoch=len(x_train) / BATCH_SIZE,
    epochs=NUM_EPOCHS,
    callbacks=[model_checkpoint, early_stopping, tensorboard_callback, reduce_lr],
    validation_data=(x_val, y_val)
)

# ---------------- Post-Training Section ----------------

# Debug print model will be saved and evaluated
print("\n=== Model Saving and Evaluation ===\n")

# Save the Trained U-Net Model
# Saving the model to disk
unet_model.save('trained_unet_model.h5')

# Debug print model has been saved successfully
print("\n=== Model Successfully Saved ===\n")



=== Model Initialization ===


=== Model Compilation ===


=== Model Training ===

Epoch 1/50
14/13 [===============================] - ETA: -8s - loss: 1.2387 - dice_coef: 0.4292 - accuracy: 0.7574 - precision: 0.3925 - recall: 0.8813
Epoch 1: saving model to unet_best_model.keras
13/13 [==============================] - 199s 15s/step - loss: 1.2387 - dice_coef: 0.4292 - accuracy: 0.7574 - precision: 0.3925 - recall: 0.8813 - val_loss: 6.1403 - val_dice_coef: 0.6566 - val_accuracy: 0.4643 - val_precision: 0.4594 - val_recall: 1.0000 - lr: 0.0010
Epoch 2/50
14/13 [===============================] - ETA: -9s - loss: 1.1017 - dice_coef: 0.4633 - accuracy: 0.8026 - precision: 0.4512 - recall: 0.8660
Epoch 2: saving model to unet_best_model.keras
13/13 [==============================] - 218s 16s/step - loss: 1.1017 - dice_coef: 0.4633 - accuracy: 0.8026 - precision: 0.4512 - recall: 0.8660 - val_loss: 5.6337 - val_dice_coef: 0.6713 - val_accuracy: 0.6050 - val_precision: 0.5354 - val_reca

Image loading, ROI and cropping 

In [6]:
# Required Libraries
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import img_to_array

# Set random seed for reproducibility
tf.random.set_seed(seed)
np.random.seed(seed)

# Load the UNet model
unet_model = tf.keras.models.load_model('trained_unet_model.h5', compile=False)

def load_images_and_labels(folders):
    """
    Load original images and labels from the specified folders.
    """
    original_images = []
    labels = []
    
    # Iterate through each folder to load images
    for folder in folders:
        standardized_folder = os.path.normpath(folder)
        print(f"Loading images from {standardized_folder}")  # Debug print
        
        for filename in os.listdir(standardized_folder):
            if filename.endswith('.jpg'):
                img = cv2.imread(os.path.join(standardized_folder, filename))
                if img is not None:
                    original_images.append(img)
                    label = standardized_folder.split(os.sep)[-1]
                    labels.append(label)
                    
    print("Loaded labels:", np.unique(labels))  # Debug print
    return original_images, labels

def find_centre_and_crop(mask, image):
    """
    Find the center of the mask and crop the image around it.
    """
    # Calculate the moments to find the centroid of the mask
    M = cv2.moments(mask)
    
    # If the mask is entirely black, return the original image
    if M["m00"] == 0:
        return image
    
    cX = int(M["m10"] / M["m00"])
    cY = int(M["m01"] / M["m00"])

    # Map the center coordinates to the original image dimensions
    original_size = image.shape[:2]
    cX = int(cX * original_size[1] / IMG_SIZE)
    cY = int(cY * original_size[0] / IMG_SIZE)

    # Target crop size
    target_size = min(original_size) * 87 // 100

    top = max(0, cY - target_size // 2)
    left = max(0, cX - target_size // 2)

    # Crop the image
    cropped_image = image[top:top + target_size, left:left + target_size]
    
    return cropped_image

def preprocess_image(image):
    """
    Preprocess the image to prepare it for prediction.
    """
    resized_image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
    resized_image = img_to_array(resized_image)
    resized_image = resized_image / 255.0
    resized_image = np.expand_dims(resized_image, axis=0)
    
    return resized_image

def predict_cropped_images(original_images):
    """
    Predict masks and crop the original images based on the masks.
    """
    cropped_images = []
    
    print("Predicting and cropping images...")  # Debug print
    
    for image in original_images:
        resized_image = preprocess_image(image)
        mask = unet_model.predict(resized_image)
        mask = np.squeeze(mask, axis=0)
        
        cropped_image = find_centre_and_crop(mask, image)
        cropped_images.append(cropped_image)
        
    print("Prediction and cropping complete.")  # Debug print
    
    return cropped_images


In [ ]:
# Required Libraries
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

# Function to display images and overlays
def display_samples(predictions, original_images, random_indices, num_samples=5):
    """Display original and masked images.

    Parameters:
    predictions (array): The predicted masks from the U-Net model.
    original_images (array): The original validation images.
    random_indices (array): The indices to sample from the validation set.
    num_samples (int): Number of random samples to display.
    """
    
    plt.figure(figsize=(15, num_samples * 5))

    for i, idx in enumerate(random_indices):
        # Display original unprocessed image
        plt.subplot(num_samples, 2, i * 2 + 1)
        plt.imshow(original_images[idx])
        plt.title(f"Sample {i+1}: Original Image")
        
        # Create and display a viridis background
        plt.subplot(num_samples, 2, i * 2 + 2)
        viridis_bg = np.zeros((original_images[idx].shape[0], original_images[idx].shape[1]))
        plt.imshow(viridis_bg, cmap='viridis')
        
        # Overlay the predicted mask on the viridis background
        overlay = np.ma.masked_where(predictions[i].squeeze() < 0.5, predictions[i].squeeze())
        plt.imshow(overlay, cmap='viridis', alpha=0.6)
        plt.title(f"Sample {i+1}: Mask Overlay")

    plt.tight_layout()
    plt.show()


# Visualize U-Net Architecture
print("Visualizing U-Net Architecture...")
tf.keras.utils.plot_model(unet_model, to_file='unet_model.png', show_shapes=True, show_layer_names=True)

# Generate Predictions on Validation Data
print("Generating predictions on validation data...")
random_indices = np.random.choice(len(x_val), size=5, replace=False)  # 5 random samples
predictions = unet_model.predict(x_val[random_indices])

# Display Random Sample Input Images and Their Corresponding Masks
print("Displaying random sample input images and their corresponding masks...")
display_samples(predictions, x_val_original, random_indices)

print("Visualization complete.")


In [8]:
import matplotlib.pyplot as plt  # Required for plotting

# Load all images and labels from specified directories
folders = ['./Cassava/brown_streak/', './Cassava/healthy1', './Cassava/mosaic_disease1']
try:
    print("Starting to load images and labels...")  # Debug print
    original_images, labels = load_images_and_labels(folders)
    print("Images and labels loaded successfully.")  # Debug print
    
    print("Starting to predict and crop images...")  # Debug print
    cropped_images = predict_cropped_images(original_images)
    print("Prediction and cropping completed successfully.")  # Debug print
except Exception as e:
    print(f"An error occurred: {e}")  # Error message
    cropped_images = []


Starting to load images and labels...
Loading images from Cassava\brown_streak


Loading images from Cassava\healthy1
Loading images from Cassava\mosaic_disease1
Loaded labels: ['brown_streak' 'healthy1' 'mosaic_disease1']
Images and labels loaded successfully.
Starting to predict and crop images...
Predicting and cropping images...
1/1 [==============================] - 0s 141ms/step
Prediction and cropping complete.
Prediction and cropping completed successfully.


In [ ]:
# Number of random samples to display
num_samples = 5

# Randomly select indices for display
print("Randomly selecting images for display...")  # Debug print
random_indices = np.random.choice(len(original_images), size=num_samples - 1, replace=False)

# Add 497 to the random indices, ensuring it's always sampled (cassava stem)
random_indices = np.append(random_indices, 497)

# Initialize the plotting area
plt.figure(figsize=(15, num_samples * 5))

# Iterate through each randomly selected index
for i, idx in enumerate(random_indices):
    # Display original image
    plt.subplot(num_samples, 2, i * 2 + 1)
    plt.imshow(cv2.cvtColor(original_images[idx], cv2.COLOR_BGR2RGB))
    plt.title(f"Original Image: {labels[idx]}")
    
    # Display the corresponding cropped image
    plt.subplot(num_samples, 2, i * 2 + 2)
    plt.imshow(cv2.cvtColor(cropped_images[idx], cv2.COLOR_BGR2RGB), cmap='viridis')
    plt.title(f"Cropped (ROI) Image: {labels[idx]}")

# Adjust layout and display the plot
plt.tight_layout()
plt.show()
print("Displaying the selected original and cropped images.")  # Debug print


In [ ]:
# Number of random samples to display
num_samples = 10

# Randomly select indices for display
print("Randomly selecting images for display...")  # Debug print
random_indices = np.random.choice(len(original_images), size=num_samples, replace=False)

# Initialize the plotting area
plt.figure(figsize=(15, num_samples * 5))

# Iterate through each randomly selected index
for i, idx in enumerate(random_indices):
    # Display original image
    plt.subplot(num_samples, 2, i * 2 + 1)
    plt.imshow(cv2.cvtColor(original_images[idx], cv2.COLOR_BGR2RGB))
    plt.title(f"Original Image: {labels[idx]}")
    
    # Display the corresponding cropped image
    plt.subplot(num_samples, 2, i * 2 + 2)
    plt.imshow(cv2.cvtColor(cropped_images[idx], cv2.COLOR_BGR2RGB), cmap='viridis')
    plt.title(f"Cropped (ROI) Image: {labels[idx]}")

# Adjust layout and display the plot
plt.tight_layout()
plt.show()
print("Displaying the selected original and cropped images.")  # Debug print


# Simple CNN

A simple CNN to establish baseline performance.

In [9]:
# Required Libraries
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import TensorBoard
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import datetime
import numpy as np  # If not already imported and loading u-net model
import cv2  # If not already imported and loading u-net model

# Load and crop images
print("Loading and cropping images...")  # Debug print
folders = ['./Cassava/brown_streak/', './Cassava/healthy1', './Cassava/mosaic_disease1']
original_images, labels = load_images_and_labels(folders)
cropped_images = predict_cropped_images(original_images)

# Preprocess images for model training
print("Preprocessing images...")  # Debug print
cropped_images_array = np.array([cv2.resize(img, (IMG_SIZE, IMG_SIZE)) for img in cropped_images]) / 255.0

# Encode labels into integers and then into one-hot encoding
print("Encoding labels...")  # Debug print
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)
categorical_labels = to_categorical(encoded_labels)

# Split the dataset into training and validation sets
print("Splitting dataset...")  # Debug print
X_train, X_val, y_train, y_val = train_test_split(cropped_images_array, categorical_labels, test_size=0.2, random_state=seed)

# Define the Convolutional Neural Network (CNN) model architecture
print("Defining model architecture...")  # Debug print
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

# Compile the model
print("Compiling model...")  # Debug print
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print model summary
model.summary()

# Set up TensorBoard for real-time performance monitoring
log_dir = "logs/fit/seq cnn simple " + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

# Train the CNN model
print("Starting model training...")  # Debug print
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, callbacks=[tensorboard_callback])
print("Model training completed.")  # Debug print


Loading and cropping images...
Loading images from Cassava\brown_streak


Loading images from Cassava\healthy1
Loading images from Cassava\mosaic_disease1
Loaded labels: ['brown_streak' 'healthy1' 'mosaic_disease1']
Predicting and cropping images...
1/1 [==============================] - 0s 141ms/step
Prediction and cropping complete.
Preprocessing images...
Encoding labels...
Splitting dataset...
Defining model architecture...
Compiling model...
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 242, 242, 32)      896       
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 121, 121, 32)      0         
 g2D)                                                            
                                                                 
 conv2d_7 (Conv2D)           (None, 119, 119, 64)      18496     
                                                                 
 max_pool

In [ ]:
# Required Libraries
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt

# ---- Evaluation Section ----

print("\n=== Model Evaluation ===\n")

# Evaluate Model on Validation Data
loss, accuracy = model.evaluate(X_val, y_val)
print(f"Validation Loss: {loss}")
print(f"Validation Accuracy: {accuracy * 100:.2f}%\n")

# Generate and Display Confusion Matrix
y_pred = model.predict(X_val)
y_pred_classes = np.argmax(y_pred, axis=1)  # Convert softmax output to label index
y_true_classes = np.argmax(y_val, axis=1)  # Convert one-hot encoded ground truth to label index
cm = confusion_matrix(y_true_classes, y_pred_classes)

print("Confusion Matrix:")
print(cm)
print("\n")

# Generate and Display Classification Report
report = classification_report(y_true_classes, y_pred_classes, target_names=label_encoder.classes_)
print("Classification Report:")
print(report)

# ---- Visualization Section ----

print("\n=== Training and Validation Curves ===\n")

# Initialize Plot
fig, axes = plt.subplots(1, 2, figsize=(12, 6))

# Plot Accuracy Curves
axes[0].plot(history.history['accuracy'], 'b--', marker='o', label='Training Accuracy', color='b') 
axes[0].plot(history.history['val_accuracy'], 'r-', marker='x', label='Validation Accuracy', color='r') 

axes[0].set_title('Training vs Validation Accuracy')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Accuracy')
axes[0].legend()

# Plot Loss Curves
axes[1].plot(history.history['loss'], 'g--', marker='o', label='Training Loss', color='g')  
axes[1].plot(history.history['val_loss'], 'm-', marker='x', label='Validation Loss', color='m')  

axes[1].set_title('Training vs Validation Loss')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Loss')
axes[1].legend()

# Show the plot
plt.tight_layout()
plt.show()

The model initially struggles to perform well on training and validation data but gradually improves. Whilst initially promising with an accuracy of 82%, further inspection shows the brown streak disease has an F1-score of 0.

# CNN with class weights

Addition of balanced class weights to address the class imbalance and boost the F1-score for the brown streak disease.

In [11]:
# Required Libraries
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import TensorBoard
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import datetime
import numpy as np
import cv2
from sklearn.utils.class_weight import compute_class_weight

# ---- Load Data ----
# Load and crop images from specified folders
print("Loading and cropping images...")
folders = ['./Cassava/brown_streak/', './Cassava/healthy1', './Cassava/mosaic_disease1']
original_images, labels = load_images_and_labels(folders)
cropped_images = predict_cropped_images(original_images)

# Convert cropped images into a numpy array suitable for training
print("Preparing images for training...")
cropped_images_array = np.array([cv2.resize(img, (IMG_SIZE, IMG_SIZE)) for img in cropped_images]) / 255.0

# ---- Preprocessing ----
# Encode labels to integers and then to categorical format
print("Encoding labels...")
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)
categorical_labels = to_categorical(encoded_labels)

# Split the dataset into training and validation sets with stratification
print("Splitting dataset into training and validation sets...")
X_train, X_val, y_train, y_val = train_test_split(
    cropped_images_array, categorical_labels, test_size=0.2, 
    random_state=seed, stratify=categorical_labels)

# ---- Model Definition ----
# Define the architecture of the Convolutional Neural Network (CNN)
print("Defining the CNN model...")
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

# Compute class weights to handle class imbalance
print("Computing class weights...")
class_weights = compute_class_weight(class_weight='balanced', 
                                     classes=np.unique(labels), y=labels)
class_weight_dict = {i : class_weights[i] for i in range(len(class_weights))}

# Compile and train the model
print("Compiling and training the model...")
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Model Summary
model.summary()

# Define TensorBoard callback for real-time performance monitoring
log_dir = "logs/fit/seq cnn plus + class weights " + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

# Train the model
history = model.fit(
    X_train, y_train, validation_data=(X_val, y_val), epochs=10, 
    callbacks=[tensorboard_callback], class_weight=class_weight_dict)


Loading and cropping images...
Loading images from Cassava\brown_streak
Loading images from Cassava\healthy1
Loading images from Cassava\mosaic_disease1
Loaded labels: ['brown_streak' 'healthy1' 'mosaic_disease1']
Predicting and cropping images...
1/1 [==============================] - 0s 129ms/step
Prediction and cropping complete.
Preparing images for training...
Encoding labels...
Splitting dataset into training and validation sets...
Defining the CNN model...
Computing class weights...
Compiling and training the model...
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_9 (Conv2D)           (None, 242, 242, 32)      896       
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 121, 121, 32)      0         
 g2D)                                                            
                                                   

In [ ]:
# Required Libraries
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt

# ---- Evaluation Section ----

print("\n=== Model Evaluation ===\n")

# Evaluate Model on Validation Data
loss, accuracy = model.evaluate(X_val, y_val)
print(f"Validation Loss: {loss}")
print(f"Validation Accuracy: {accuracy * 100:.2f}%\n")

# Generate and Display Confusion Matrix
y_pred = model.predict(X_val)
y_pred_classes = np.argmax(y_pred, axis=1)  # Convert softmax output to label index
y_true_classes = np.argmax(y_val, axis=1)  # Convert one-hot encoded ground truth to label index
cm = confusion_matrix(y_true_classes, y_pred_classes)

print("Confusion Matrix:")
print(cm)
print("\n")

# Generate and Display Classification Report
report = classification_report(y_true_classes, y_pred_classes, target_names=label_encoder.classes_)
print("Classification Report:")
print(report)

# ---- Visualization Section ----

print("\n=== Training and Validation Curves ===\n")

# Initialize Plot
fig, axes = plt.subplots(1, 2, figsize=(12, 6))

# Plot Accuracy Curves
axes[0].plot(history.history['accuracy'], 'b--', marker='o', label='Training Accuracy', color='b') 
axes[0].plot(history.history['val_accuracy'], 'r-', marker='x', label='Validation Accuracy', color='r') 

axes[0].set_title('Training vs Validation Accuracy')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Accuracy')
axes[0].legend()

# Plot Loss Curves
axes[1].plot(history.history['loss'], 'g--', marker='o', label='Training Loss', color='g')  
axes[1].plot(history.history['val_loss'], 'm-', marker='x', label='Validation Loss', color='m')  

axes[1].set_title('Training vs Validation Loss')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Loss')
axes[1].legend()

# Show the plot
plt.tight_layout()
plt.show()

Though overall accuracy is down to 59%, the smallest class (brown streak disease) sees an improved F1 Score of 35%, and the persistent gaps between training and validation accuracies suggest overfitting, highlighting the need for further optimization to enhance generalization.

# CNN with K-Fold

Stratified K-Fold Cross-Validation is used with the current CNN to maximise all the training data.

In [ ]:
# Required Libraries
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import TensorBoard
import datetime
import cv2  
import pandas as pd
import matplotlib.cm as cm

# Setting the seed for reproducibility
np.random.seed(seed)
tf.random.set_seed(seed)

# Define function to plot metrics
def plot_metrics(metrics, metric_name):
    labels = list(metrics.keys())
    values = [np.mean(metrics[label]) for label in labels]
    
    # Generate colors from viridis colormap within the range [4/4.5, 4.5/7]
    colormap = cm.viridis(np.linspace(4/4.5, 4.5/7, len(labels)))
    
    plt.figure(figsize=(10, 6))
    plt.barh(labels, values, color=colormap)  # Use colors from the adjusted viridis colormap
    plt.xlabel(metric_name)
    plt.ylabel('Class Labels')
    plt.title(f'Average {metric_name} for Each Class')
    for i, v in enumerate(values):
        plt.text(v, i, str(round(v, 2)), va='center', ha="left")
    plt.show()

# Initialize metrics dictionary for precision, recall, F1 score, and accuracy
class_metrics = {'precision': {}, 'recall': {}, 'f1-score': {}, 'accuracy': {}}

# Loading and preprocessing images
folders = ['./Cassava/brown_streak/', './Cassava/healthy1', './Cassava/mosaic_disease1']
original_images, labels = load_images_and_labels(folders)
cropped_images = predict_cropped_images(original_images)
cropped_images_array = np.array([cv2.resize(img, (IMG_SIZE, IMG_SIZE)) for img in cropped_images])

# Encoding labels
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)
categorical_labels = to_categorical(encoded_labels)

# K-Folds Cross-Validation setup
stratified_k_fold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
class_weights = compute_class_weight('balanced', classes=np.unique(labels), y=labels)
class_weight_dict = {i: class_weights[i] for i in range(len(class_weights))}

confusion_matrices = []

# K-Fold Cross-Validation loop
for train_index, val_index in stratified_k_fold.split(cropped_images_array, encoded_labels):

    X_train, X_test = cropped_images_array[train_index], cropped_images_array[val_index]
    y_train, y_test = categorical_labels[train_index], categorical_labels[val_index]

    # CNN model definition
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(512, activation='relu'),
        Dropout(0.5),
        Dense(len(label_encoder.classes_), activation='softmax')
    ])

    # Compile and train the model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    log_dir = "logs/cnn + weights n kfold" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)
    model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, class_weight=class_weight_dict,
               callbacks=[tensorboard_callback])

    # Model evaluation and metrics calculation
    y_pred = model.predict(X_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_true_classes = np.argmax(y_test, axis=1)
    report = classification_report(y_true_classes, y_pred_classes, output_dict=True, zero_division=1)

    cnf_matrix = confusion_matrix(y_true_classes, y_pred_classes)
    confusion_matrices.append(cnf_matrix)

    for lbl in label_encoder.classes_:
        class_idx = label_encoder.transform([lbl])[0]
        class_metrics['precision'][lbl] = class_metrics['precision'].get(lbl, []) + [report[str(class_idx)]['precision']]
        class_metrics['recall'][lbl] = class_metrics['recall'].get(lbl, []) + [report[str(class_idx)]['recall']]
        class_metrics['f1-score'][lbl] = class_metrics['f1-score'].get(lbl, []) + [report[str(class_idx)]['f1-score']]
        class_metrics['accuracy'][lbl] = class_metrics['accuracy'].get(lbl, []) + [accuracy_score(y_true_classes[y_true_classes==class_idx],
                                                                                                                           y_pred_classes[y_true_classes==class_idx])]

# Averaging the metrics
for metric, class_data in class_metrics.items():
    for lbl, values in class_data.items():
        class_metrics[metric][lbl] = np.mean(values)

# Visualizing the metrics
for metric_name in ['f1-score', 'accuracy']:
    plot_metrics(class_metrics[metric_name], metric_name)

# Sample three images from each class/label
sample_images = []
sample_labels = []

for lbl in label_encoder.classes_:
    class_idx = label_encoder.transform([lbl])[0]
    indices = np.where(np.array(y_true_classes) == class_idx)[0][:3]  # Sample three images per class
    sample_images.extend(X_test[indices])
    sample_labels.extend([lbl] * 3)

# Generate predictions for the sampled images
sample_predictions = model.predict(np.array(sample_images))
sample_predictions = np.argmax(sample_predictions, axis=1)

# Decode the true and predicted labels to their original string labels
decoded_true_labels = label_encoder.inverse_transform([label_encoder.transform([lbl])[0] for lbl in sample_labels])
decoded_predicted_labels = label_encoder.inverse_transform(sample_predictions)

# Plot the sampled images along with their true and predicted labels
plt.figure(figsize=(15, 15))
for i in range(len(sample_images)):
    plt.subplot(3, 3, i+1)
    plt.imshow(sample_images[i])
    plt.title(f"True: {decoded_true_labels[i]}\nPred: {decoded_predicted_labels[i]}", fontsize=10)
    plt.axis('off')
plt.tight_layout()
plt.show()

Using the Stratified K-Fold Cross-Validation had varying performance levels across different leaf conditions. It performed relatively well for "healthy" leaves, with a high average F1 score and accuracy. The "mosaic disease" class showed a reasonable F1 score and accuracy. However, the "brown streak" class had lower F1-score and accuracy values, indicating that the model had more difficulty classifying this condition accurately.

# Data Augumentation

Data augmentation to mitigate and improve the F1 scores

In [14]:
# Required Libraries
import numpy as np
import tensorflow as tf
import cv2
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import datetime
from sklearn.metrics import accuracy_score
import pandas as pd
import seaborn as sns

# Setting the seed for reproducibility
np.random.seed(seed)
tf.random.set_seed(seed)

# Load and crop images
folders = ['./Cassava/brown_streak/', './Cassava/healthy1', './Cassava/mosaic_disease1']
original_images, labels = load_images_and_labels(folders)

# Convert cropped images into a format suitable for training
cropped_images_array = np.array([cv2.resize(img, (224, 224)) for img in original_images]) / 255.0

# Encode labels
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)
categorical_labels = to_categorical(encoded_labels)

# Create a StratifiedKFold object
stratified_k_fold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)

# Define class weights
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(labels), y=labels)
class_weight_dict = {i: class_weights[i] for i in range(len(class_weights))}

# Create a data augmentation generator
train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    shear_range=0.2,
    vertical_flip=True,
    horizontal_flip=True,
    fill_mode='nearest')

# Compute confusion matrices for each class
confusion_matrices = []

# Initialize metrics dictionary for precision, recall, F1 score, and accuracy
class_metrics = {'precision': {}, 'recall': {}, 'f1-score': {}, 'accuracy': {}}

# K-Fold Cross-Validation loop
for train_index, val_index in stratified_k_fold.split(cropped_images_array, encoded_labels):
    
    X_train, X_val = cropped_images_array[train_index], cropped_images_array[val_index]
    y_train, y_val = categorical_labels[train_index], categorical_labels[val_index]
    
    # Create data generator for the training set
    train_generator = train_datagen.flow(X_train, y_train, batch_size=32)
    
    # Define the CNN model
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(512, activation='relu'),
        Dropout(0.5),
        Dense(len(label_encoder.classes_), activation='softmax')
    ])
    
    # Compile the model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    # Define TensorBoard callback
    log_dir = "logs/fit/ CNN with augmentation" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)
    
    # Fit the model using the data generator
    history = model.fit(train_generator, validation_data=(X_val, y_val), epochs=10, callbacks=[tensorboard_callback],
                         class_weight=class_weight_dict)
    
    y_pred = model.predict(X_val)
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_true = np.argmax(y_val, axis=1)
    
    # Compute confusion matrix and add to the list
    cnf_matrix = confusion_matrix(y_true, y_pred_classes)
    confusion_matrices.append(cnf_matrix)

    # Calculate and accumulate the metrics
    report = classification_report(y_true, y_pred_classes, output_dict=True, zero_division=1)

    for lbl in label_encoder.classes_:
        class_idx = label_encoder.transform([lbl])[0]
        class_metrics['precision'][lbl] = class_metrics['precision'].get(lbl, []) + [report[str(class_idx)]['precision']]
        class_metrics['recall'][lbl] = class_metrics['recall'].get(lbl, []) + [report[str(class_idx)]['recall']]
        class_metrics['f1-score'][lbl] = class_metrics['f1-score'].get(lbl, []) + [report[str(class_idx)]['f1-score']]
        class_metrics['accuracy'][lbl] = class_metrics['accuracy'].get(lbl, []) + [accuracy_score(y_true[y_true==class_idx], 
                                                                                                  y_pred_classes[y_true==class_idx])]
    
    # Compute confusion matrix and add to the list
    cnf_matrix = confusion_matrix(y_true, y_pred_classes)
    confusion_matrices.append(cnf_matrix)



Loading images from Cassava\brown_streak
Loading images from Cassava\healthy1
Loading images from Cassava\mosaic_disease1
Loaded labels: ['brown_streak' 'healthy1' 'mosaic_disease1']
Epoch 1/10
14/14 [==============================] - 34s 2s/step - loss: 1.9210 - accuracy: 0.4182 - val_loss: 1.1546 - val_accuracy: 0.2778
Epoch 2/10
14/14 [==============================] - 19s 1s/step - loss: 0.9133 - accuracy: 0.5070 - val_loss: 0.9955 - val_accuracy: 0.5463
Epoch 3/10
14/14 [==============================] - 19s 1s/step - loss: 0.7343 - accuracy: 0.5981 - val_loss: 1.2468 - val_accuracy: 0.6204
Epoch 4/10
14/14 [==============================] - 18s 1s/step - loss: 0.8144 - accuracy: 0.5467 - val_loss: 0.8014 - val_accuracy: 0.6389
Epoch 5/10
14/14 [==============================] - 17s 1s/step - loss: 0.5883 - accuracy: 0.7266 - val_loss: 0.5555 - val_accuracy: 0.7500
Epoch 6/10
14/14 [==============================] - 17s 1s/step - loss: 0.5485 - accuracy: 0.7033 - val_loss: 0.6026 

In [ ]:
# Averaging the metrics
for metric, class_data in class_metrics.items():
    for lbl, values in class_data.items():
        class_metrics[metric][lbl] = np.mean(values)

# Visualizing the metrics
for metric_name in ['f1-score', 'accuracy']:
    plot_metrics(class_metrics[metric_name], metric_name)

# Sample three images from each class/label
sample_images = []
sample_labels = []

for lbl in label_encoder.classes_:
    class_idx = label_encoder.transform([lbl])[0]
    indices = np.where(np.array(y_true) == class_idx)[0][:3]  # Sample three images per class
    sample_images.extend(X_test[indices])
    sample_labels.extend([lbl] * 3)

# Resize sampled images to match the model's input shape
sample_images_resized = [cv2.resize(img, (224, 224)) for img in sample_images]

# Generate predictions for the sampled images
sample_predictions = model.predict(np.array(sample_images_resized))
sample_predictions = np.argmax(sample_predictions, axis=1)

# Decode the true and predicted labels to their original string labels
decoded_true_labels = label_encoder.inverse_transform([label_encoder.transform([lbl])[0] for lbl in sample_labels])
decoded_predicted_labels = label_encoder.inverse_transform(sample_predictions)

# Plot the sampled images along with their true and predicted labels
plt.figure(figsize=(15, 15))
for i in range(len(sample_images)):
    plt.subplot(3, 3, i+1)
    plt.imshow(sample_images[i])
    plt.title(f"True: {decoded_true_labels[i]}\nPred: {decoded_predicted_labels[i]}", fontsize=10)
    plt.axis('off')
plt.tight_layout()
plt.show()

After applying data augmentation techniques, the model's performance significantly improved across all three classes. For mosaic diseases and healthy leaf conditions, both F1-score and accuracy substantially increased, indicating a better ability to classify these conditions. However, for the brown streak, this class remains challenging for the model even after augmentation.

# CNN with SMOTE and adjusted decision threshold

The focus is to address the regression observed in the previous model; a lowered decision threshold alongside applying SMOTE to the CNN may reverse and improve the F1-score of the underrepresented class, brown streak disease.

In [16]:
# Required Libraries
import numpy as np
import tensorflow as tf
import cv2
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import datetime
from imblearn.over_sampling import SMOTE

# Custom decision threshold
OVERROLL_THRESHOLD = 0.4

# Setting the seed for reproducibility
np.random.seed(seed)
tf.random.set_seed(seed)

# Load and crop images
folders = ['./Cassava/brown_streak/', './Cassava/healthy1', './Cassava/mosaic_disease1']
original_images, labels = load_images_and_labels(folders)

# Convert cropped images into a format suitable for training
cropped_images_array = np.array([cv2.resize(img, (224, 224)) for img in original_images]) / 255.0

# Encode labels
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)
categorical_labels = to_categorical(encoded_labels)

# Create a StratifiedKFold object
stratified_k_fold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)

# Define class weights
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(labels), y=labels)
class_weight_dict = {i: class_weights[i] for i in range(len(class_weights))}

# Create a data augmentation generator
train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    shear_range=0.2,
    vertical_flip=True,
    horizontal_flip=True,
    fill_mode='nearest')

# Compute confusion matrices for each class
confusion_matrices = []

# Initialize metrics dictionary for precision, recall, F1 score, and accuracy
class_metrics = {'precision': {}, 'recall': {}, 'f1-score': {}, 'accuracy': {}}

# K-Fold Cross-Validation loop
for train_index, val_index in stratified_k_fold.split(cropped_images_array, encoded_labels):
    
    X_train, X_val = cropped_images_array[train_index], cropped_images_array[val_index]
    y_train, y_val = categorical_labels[train_index], categorical_labels[val_index]
        
    # Apply SMOTE to the training data only
    smote = SMOTE()
    X_train_reshaped = X_train.reshape(X_train.shape[0], -1)
    X_train_smote, y_train_smote = smote.fit_resample(X_train_reshaped, y_train)
    X_train_smote = X_train_smote.reshape(X_train_smote.shape[0], 224, 224, 3)
    
    # Create data generator for the training set
    train_generator = train_datagen.flow(X_train_smote, y_train_smote,batch_size=BATCH_SIZE, seed=seed)
    
    # Define the CNN model
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(512, activation='relu'),
        Dropout(0.5),
        Dense(len(label_encoder.classes_), activation='softmax')
    ])
    
    # Compile the model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    # Define TensorBoard callback
    log_dir = "logs/fit/CNN with SMOTE" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)
    
    # Fit the model using the data generator
    history = model.fit(train_generator, validation_data=(X_val, y_val), epochs=10, callbacks=[tensorboard_callback],
                         class_weight=class_weight_dict)
    
    y_pred = model.predict(X_val)
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_true = np.argmax(y_val, axis=1)

    # Apply custom decision threshold
    y_pred_binary = (y_pred > OVERROLL_THRESHOLD).astype(int)
    
    # Compute confusion matrix and add to the list
    cnf_matrix = confusion_matrix(y_true, y_pred_classes)
    confusion_matrices.append(cnf_matrix)

    # Calculate and accumulate the metrics
    report = classification_report(y_true, y_pred_classes, output_dict=True, zero_division=1)
    
    for lbl in label_encoder.classes_:
        class_idx = label_encoder.transform([lbl])[0]
        class_metrics['precision'][lbl] = class_metrics['precision'].get(lbl, []) + [report[str(class_idx)]['precision']]
        class_metrics['recall'][lbl] = class_metrics['recall'].get(lbl, []) + [report[str(class_idx)]['recall']]
        class_metrics['f1-score'][lbl] = class_metrics['f1-score'].get(lbl, []) + [report[str(class_idx)]['f1-score']]
        class_metrics['accuracy'][lbl] = class_metrics['accuracy'].get(lbl, []) + [accuracy_score(y_true[y_true==class_idx],
                                                                                                   y_pred_classes[y_true==class_idx])]



Loading images from Cassava\brown_streak
Loading images from Cassava\healthy1
Loading images from Cassava\mosaic_disease1
Loaded labels: ['brown_streak' 'healthy1' 'mosaic_disease1']
Epoch 1/10
26/26 [==============================] - 37s 1s/step - loss: 3.0905 - accuracy: 0.4453 - val_loss: 1.3336 - val_accuracy: 0.3333
Epoch 2/10
26/26 [==============================] - 28s 1s/step - loss: 0.7457 - accuracy: 0.6468 - val_loss: 0.6162 - val_accuracy: 0.7315
Epoch 3/10
26/26 [==============================] - 27s 1s/step - loss: 0.5293 - accuracy: 0.7500 - val_loss: 0.5267 - val_accuracy: 0.8426
Epoch 4/10
26/26 [==============================] - 30s 1s/step - loss: 0.6808 - accuracy: 0.7388 - val_loss: 1.8498 - val_accuracy: 0.3796
Epoch 5/10
26/26 [==============================] - 31s 1s/step - loss: 0.4982 - accuracy: 0.7836 - val_loss: 0.7724 - val_accuracy: 0.7407
Epoch 6/10
26/26 [==============================] - 29s 1s/step - loss: 0.5296 - accuracy: 0.7749 - val_loss: 0.3978 

In [ ]:
# Averaging the metrics
for metric, class_data in class_metrics.items():
    for lbl, values in class_data.items():
        class_metrics[metric][lbl] = np.mean(values)

# Visualizing the metrics
for metric_name in ['f1-score', 'accuracy']:
    plot_metrics(class_metrics[metric_name], metric_name)

# Sample three images from each class/label
sample_images = []
sample_labels = []

for lbl in label_encoder.classes_:
    class_idx = label_encoder.transform([lbl])[0]
    indices = np.where(np.array(y_true) == class_idx)[0][:3]  # Sample three images per class
    sample_images.extend(X_test[indices])
    sample_labels.extend([lbl] * 3)

# Resize sampled images to match the model's input shape
sample_images_resized = [cv2.resize(img, (224, 224)) for img in sample_images]

# Generate predictions for the sampled images with the same thresholds as during validation
sample_predictions = model.predict(np.array(sample_images_resized))

# Apply custom decision thresholds
brown_streak_index = label_encoder.transform(['brown_streak'])[0]  # Get the index for the 'brown_streak' class
sample_predictions_binary = np.zeros_like(sample_predictions)
sample_predictions_binary[sample_predictions > OVERROLL_THRESHOLD] = 1  
sample_predictions_binary[(sample_predictions[:, brown_streak_index] > OVERROLL_THRESHOLD), brown_streak_index] = 1  

# Convert to class labels based on custom decision thresholds
sample_predictions_classes = np.argmax(sample_predictions_binary, axis=1)

# Decode the true and predicted labels to their original string labels
decoded_true_labels = label_encoder.inverse_transform([label_encoder.transform([lbl])[0] for lbl in sample_labels])
decoded_predicted_labels = label_encoder.inverse_transform(sample_predictions_classes)

# Plot the sampled images along with their true and predicted labels
plt.figure(figsize=(15, 15))
for i in range(len(sample_images)):
    plt.subplot(3, 3, i+1)
    plt.imshow(sample_images[i])
    plt.title(f"True: {decoded_true_labels[i]}\nPred: {decoded_predicted_labels[i]}", fontsize=10)
    plt.axis('off')
plt.tight_layout()
plt.show()


These enhancements suggest that combining data augmentation, SMOTE, and adjusting the decision threshold has improved the model's ability to classify the different leaf conditions, with particular improvements in the mosaic disease and healthy classes; brown streak also showed improvement in F1-score and accuracy, though it remains a more challenging class to classify accurately compared to the other two.

#### The brown streak disease has only 50 image samples, so its class's decision threshold is reduced further to 0.35.

In [20]:
# Required Libraries
import numpy as np
import tensorflow as tf
import cv2
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import datetime
from imblearn.over_sampling import SMOTE

# Custom decision threshold
OVERROLL_THRESHOLD = 0.4
BROWN_STREAK_THRESHOLD = 0.35

# Setting the seed for reproducibility
np.random.seed(seed)
tf.random.set_seed(seed)

# Load and crop images
folders = ['./Cassava/brown_streak/', './Cassava/healthy1', './Cassava/mosaic_disease1']
original_images, labels = load_images_and_labels(folders)

# Convert cropped images into a format suitable for training
cropped_images_array = np.array([cv2.resize(img, (224, 224)) for img in original_images]) / 255.0

# Encode labels
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)
categorical_labels = to_categorical(encoded_labels)

# Create a StratifiedKFold object
stratified_k_fold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)

# Define class weights
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(labels), y=labels)
class_weight_dict = {i: class_weights[i] for i in range(len(class_weights))}

# Create a data augmentation generator
train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    shear_range=0.2,
    vertical_flip=True,
    horizontal_flip=True,
    fill_mode='nearest')

# Compute confusion matrices for each class
confusion_matrices = []

# Initialize metrics dictionary for precision, recall, F1 score, and accuracy
class_metrics = {'precision': {}, 'recall': {}, 'f1-score': {}, 'accuracy': {}}

# K-Fold Cross-Validation loop
for train_index, val_index in stratified_k_fold.split(cropped_images_array, encoded_labels):
    
    X_train, X_val = cropped_images_array[train_index], cropped_images_array[val_index]
    y_train, y_val = categorical_labels[train_index], categorical_labels[val_index]    
    
    # Apply SMOTE to the training data only
    smote = SMOTE()
    X_train_reshaped = X_train.reshape(X_train.shape[0], -1)
    X_train_smote, y_train_smote = smote.fit_resample(X_train_reshaped, y_train)
    X_train_smote = X_train_smote.reshape(X_train_smote.shape[0], 224, 224, 3)
    
    # Create data generator for the training set
    train_generator = train_datagen.flow(X_train_smote, y_train_smote,batch_size=BATCH_SIZE, seed=seed)
    
    # Define the CNN model
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(512, activation='relu'),
        Dropout(0.5),
        Dense(len(label_encoder.classes_), activation='softmax')
    ])
    
    # Compile the model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    # Define TensorBoard callback
    log_dir = "logs/fit/smote and threshold for brown streak" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)
    
    # Fit the model using the data generator
    history = model.fit(train_generator, validation_data=(X_val, y_val), epochs=10, callbacks=[tensorboard_callback], 
                        class_weight=class_weight_dict)
        
    # Model evaluation and metrics calculation
    y_pred = model.predict(X_val)
    y_true = np.argmax(y_val, axis=1)
    
    # Apply custom decision thresholds
    brown_streak_index = label_encoder.transform(['brown_streak'])[0]  # Get the index for the 'brown_streak' class
    y_pred_binary = np.zeros_like(y_pred)
    y_pred_binary[y_pred > OVERROLL_THRESHOLD] = 1
    y_pred_binary[(y_pred[:, brown_streak_index] > BROWN_STREAK_THRESHOLD), brown_streak_index] = 1  # Special threshold for brown_streak
    
    # Convert to class labels based on custom decision thresholds
    y_pred_classes = np.argmax(y_pred_binary, axis=1)
            
    # Compute confusion matrix and add to the list
    cnf_matrix = confusion_matrix(y_true, y_pred_classes)
    confusion_matrices.append(cnf_matrix)

    # Calculate and accumulate the metrics
    report = classification_report(y_true, y_pred_classes, output_dict=True, zero_division=1)
    
    for lbl in label_encoder.classes_:
        class_idx = label_encoder.transform([lbl])[0]
        class_metrics['precision'][lbl] = class_metrics['precision'].get(lbl, []) + [report[str(class_idx)]['precision']]
        class_metrics['recall'][lbl] = class_metrics['recall'].get(lbl, []) + [report[str(class_idx)]['recall']]
        class_metrics['f1-score'][lbl] = class_metrics['f1-score'].get(lbl, []) + [report[str(class_idx)]['f1-score']]
        class_metrics['accuracy'][lbl] = class_metrics['accuracy'].get(lbl, []) + [accuracy_score(y_true[y_true==class_idx],
                                                                                                   y_pred_classes[y_true==class_idx])]



Loading images from Cassava\brown_streak
Loading images from Cassava\healthy1
Loading images from Cassava\mosaic_disease1
Loaded labels: ['brown_streak' 'healthy1' 'mosaic_disease1']
Epoch 1/10
26/26 [==============================] - 32s 1s/step - loss: 1.2221 - accuracy: 0.5622 - val_loss: 0.6935 - val_accuracy: 0.7222
Epoch 2/10
26/26 [==============================] - 28s 1s/step - loss: 0.6594 - accuracy: 0.7500 - val_loss: 1.1093 - val_accuracy: 0.7130
Epoch 3/10
26/26 [==============================] - 29s 1s/step - loss: 0.4821 - accuracy: 0.7935 - val_loss: 0.5115 - val_accuracy: 0.8333
Epoch 4/10
26/26 [==============================] - 29s 1s/step - loss: 0.5163 - accuracy: 0.7774 - val_loss: 0.3429 - val_accuracy: 0.8426
Epoch 5/10
26/26 [==============================] - 29s 1s/step - loss: 0.3599 - accuracy: 0.8483 - val_loss: 0.4459 - val_accuracy: 0.8333
Epoch 6/10
26/26 [==============================] - 28s 1s/step - loss: 0.3440 - accuracy: 0.8495 - val_loss: 0.3223 

In [ ]:
# Averaging the metrics
for metric, class_data in class_metrics.items():
    for lbl, values in class_data.items():
        class_metrics[metric][lbl] = np.mean(values)

# Visualizing the metrics
for metric_name in ['f1-score', 'accuracy']:
    plot_metrics(class_metrics[metric_name], metric_name)

# Sample three images from each class/label
sample_images = []
sample_labels = []

for lbl in label_encoder.classes_:
    class_idx = label_encoder.transform([lbl])[0]
    indices = np.where(np.array(y_true) == class_idx)[0][:3]  # Sample three images per class
    sample_images.extend(X_test[indices])
    sample_labels.extend([lbl] * 3)

# Resize sampled images to match the model's input shape
sample_images_resized = [cv2.resize(img, (224, 224)) for img in sample_images]

# Generate predictions for the sampled images with the same thresholds as during validation
sample_predictions = model.predict(np.array(sample_images_resized))

# Apply custom decision thresholds: 0.7 for all classes and 0.35 for brown_streak
brown_streak_index = label_encoder.transform(['brown_streak'])[0]  # Get the index for the 'brown_streak' class
sample_predictions_binary = np.zeros_like(sample_predictions)
sample_predictions_binary[sample_predictions > OVERROLL_THRESHOLD] = 1  # Higher threshold for other two classes to 0.7
sample_predictions_binary[(sample_predictions[:, brown_streak_index] > BROWN_STREAK_THRESHOLD), brown_streak_index] = 1  # Lower threshold for 'brown_streak' class to 0.4

# Convert to class labels based on custom decision thresholds
sample_predictions_classes = np.argmax(sample_predictions_binary, axis=1)

# Decode the true and predicted labels to their original string labels
decoded_true_labels = label_encoder.inverse_transform([label_encoder.transform([lbl])[0] for lbl in sample_labels])
decoded_predicted_labels = label_encoder.inverse_transform(sample_predictions_classes)

# Plot the sampled images along with their true and predicted labels
plt.figure(figsize=(15, 15))
for i in range(len(sample_images)):
    plt.subplot(3, 3, i+1)
    plt.imshow(sample_images[i])
    plt.title(f"True: {decoded_true_labels[i]}\nPred: {decoded_predicted_labels[i]}", fontsize=10)
    plt.axis('off')
plt.tight_layout()
plt.show()


With the reduced threshold for brown streak, there was a noticeable improvement in mosaic disease and healthy classes regarding F1 score and accuracy, with both classes achieving higher scores. However, the brown streak still presented difficulties, with a lower F1-score and accuracy, suggesting that this class remains challenging for classification even with these various adjustments.

# Switch to ADASYN

ADASYN is an improvement upon the SMOTE approach and is applied to address the extreme class imbalance in the dataset

In [22]:
# Required Libraries
import numpy as np
import tensorflow as tf
import cv2
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import datetime
from imblearn.over_sampling import ADASYN
from tensorflow.keras.optimizers import Adam

# Custom decision threshold
OVERROLL_THRESHOLD = 0.4
BROWN_STREAK_THRESHOLD = 0.35

# Setting the seed for reproducibility
np.random.seed(seed)
tf.random.set_seed(seed)

# Load and crop images
folders = ['./Cassava/brown_streak/', './Cassava/healthy1', './Cassava/mosaic_disease1']
original_images, labels = load_images_and_labels(folders)

# Convert cropped images into a format suitable for training
cropped_images_array = np.array([cv2.resize(img, (224, 224)) for img in original_images]) / 255.0

# Encode labels
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)
categorical_labels = to_categorical(encoded_labels)

# Create a StratifiedKFold object
stratified_k_fold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)

# Define class weights
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(labels), y=labels)
class_weight_dict = {i: class_weights[i] for i in range(len(class_weights))}

# Create a data augmentation generator
train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    shear_range=0.2,
    vertical_flip=True,
    horizontal_flip=True,
    fill_mode='nearest')

# Compute confusion matrices for each class
confusion_matrices = []

# Initialize metrics dictionary for precision, recall, F1 score, and accuracy
class_metrics = {'precision': {}, 'recall': {}, 'f1-score': {}, 'accuracy': {}}

# K-Fold Cross-Validation loop
for train_index, val_index in stratified_k_fold.split(cropped_images_array, encoded_labels):
    
    X_train, X_val = cropped_images_array[train_index], cropped_images_array[val_index]
    y_train, y_val = categorical_labels[train_index], categorical_labels[val_index]    
    
    # Apply ADASYN to the training data only
    adasyn = ADASYN()
    X_train_reshaped = X_train.reshape(X_train.shape[0], -1)
    X_train_adasyn, y_train_adasyn = adasyn.fit_resample(X_train_reshaped, y_train)
    X_train_adasyn = X_train_adasyn.reshape(X_train_adasyn.shape[0], 224, 224, 3)
    
    # Create data generator for the training set
    train_generator = train_datagen.flow(X_train_adasyn, y_train_adasyn,batch_size=BATCH_SIZE, seed=seed)
    
    # Define the enhanced CNN model
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(256, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(512, activation='relu'),
        Dropout(0.7),
        Dense(len(label_encoder.classes_), activation='softmax')
    ])

    initial_learning_rate = 0.001
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate,
        decay_steps=10000,
        decay_rate=0.9)
    optimizer = Adam(learning_rate=lr_schedule)
    
    # Compile the model
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    
    # Define TensorBoard callback
    log_dir = "logs/fit/ADASYN" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

    early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
    
    # Fit the model using the data generator
    history = model.fit(train_generator, validation_data=(X_val, y_val), epochs=25, 
                        callbacks=[tensorboard_callback,early_stopping], class_weight=class_weight_dict)
        
    # Model evaluation and metrics calculation
    y_pred = model.predict(X_val)
    y_true = np.argmax(y_val, axis=1)
    
    # Apply custom decision thresholds
    brown_streak_index = label_encoder.transform(['brown_streak'])[0]  # Get the index for the 'brown_streak' class
    y_pred_binary = np.zeros_like(y_pred)
    y_pred_binary[y_pred > OVERROLL_THRESHOLD] = 1
    y_pred_binary[(y_pred[:, brown_streak_index] > BROWN_STREAK_THRESHOLD), brown_streak_index] = 1  # Special threshold for brown_streak
    
    # Convert to class labels based on custom decision thresholds
    y_pred_classes = np.argmax(y_pred_binary, axis=1)
            
    # Compute confusion matrix and add to the list
    cnf_matrix = confusion_matrix(y_true, y_pred_classes)
    confusion_matrices.append(cnf_matrix)

    # Calculate and accumulate the metrics
    report = classification_report(y_true, y_pred_classes, output_dict=True, zero_division=1)
    
    for lbl in label_encoder.classes_:
        class_idx = label_encoder.transform([lbl])[0]
        class_metrics['precision'][lbl] = class_metrics['precision'].get(lbl, []) + [report[str(class_idx)]['precision']]
        class_metrics['recall'][lbl] = class_metrics['recall'].get(lbl, []) + [report[str(class_idx)]['recall']]
        class_metrics['f1-score'][lbl] = class_metrics['f1-score'].get(lbl, []) + [report[str(class_idx)]['f1-score']]
        class_metrics['accuracy'][lbl] = class_metrics['accuracy'].get(lbl, []) + [accuracy_score(y_true[y_true==class_idx],
                                                                                                   y_pred_classes[y_true==class_idx])]



Loading images from Cassava\brown_streak
Loading images from Cassava\healthy1
Loading images from Cassava\mosaic_disease1
Loaded labels: ['brown_streak' 'healthy1' 'mosaic_disease1']
Epoch 1/25
26/26 [==============================] - 42s 2s/step - loss: 1.5934 - accuracy: 0.4353 - val_loss: 1.3049 - val_accuracy: 0.3704
Epoch 2/25
26/26 [==============================] - 37s 1s/step - loss: 0.6797 - accuracy: 0.6771 - val_loss: 2.3998 - val_accuracy: 0.4444
Epoch 3/25
26/26 [==============================] - 37s 1s/step - loss: 0.8218 - accuracy: 0.7231 - val_loss: 0.7263 - val_accuracy: 0.7870
Epoch 4/25
26/26 [==============================] - 42s 2s/step - loss: 0.6556 - accuracy: 0.7316 - val_loss: 0.4320 - val_accuracy: 0.8241
Epoch 5/25
26/26 [==============================] - 44s 2s/step - loss: 0.5201 - accuracy: 0.7666 - val_loss: 0.5880 - val_accuracy: 0.7407
Epoch 6/25
26/26 [==============================] - 42s 2s/step - loss: 0.4281 - accuracy: 0.8198 - val_loss: 0.7353 

In [ ]:
# Averaging the metrics
for metric, class_data in class_metrics.items():
    for lbl, values in class_data.items():
        class_metrics[metric][lbl] = np.mean(values)

# Visualizing the metrics
for metric_name in ['f1-score', 'accuracy']:
    plot_metrics(class_metrics[metric_name], metric_name)

# Sample three images from each class/label
sample_images = []
sample_labels = []

for lbl in label_encoder.classes_:
    class_idx = label_encoder.transform([lbl])[0]
    indices = np.where(np.array(y_true) == class_idx)[0][:3]  # Sample three images per class
    sample_images.extend(X_test[indices])
    sample_labels.extend([lbl] * 3)

# Resize sampled images to match the model's input shape
sample_images_resized = [cv2.resize(img, (224, 224)) for img in sample_images]

# Generate predictions for the sampled images with the same thresholds as during validation
sample_predictions = model.predict(np.array(sample_images_resized))

# Apply custom decision thresholds
brown_streak_index = label_encoder.transform(['brown_streak'])[0]  # Get the index for the 'brown_streak' class
sample_predictions_binary = np.zeros_like(sample_predictions)
sample_predictions_binary[sample_predictions > OVERROLL_THRESHOLD] = 1 
sample_predictions_binary[(sample_predictions[:, brown_streak_index] > BROWN_STREAK_THRESHOLD), brown_streak_index] = 1   # Special threshold for brown_streak

# Convert to class labels based on custom decision thresholds
sample_predictions_classes = np.argmax(sample_predictions_binary, axis=1)

# Decode the true and predicted labels to their original string labels
decoded_true_labels = label_encoder.inverse_transform([label_encoder.transform([lbl])[0] for lbl in sample_labels])
decoded_predicted_labels = label_encoder.inverse_transform(sample_predictions_classes)

# Plot the sampled images along with their true and predicted labels
plt.figure(figsize=(15, 15))
for i in range(len(sample_images)):
    plt.subplot(3, 3, i+1)
    plt.imshow(sample_images[i])
    plt.title(f"True: {decoded_true_labels[i]}\nPred: {decoded_predicted_labels[i]}", fontsize=10)
    plt.axis('off')
plt.tight_layout()
plt.show()


These adjustments improved the model's ability to classify mosaic disease and healthy conditions, with brown streak still a more challenging class. However, it showed a modest improvement in F1 score and accuracy.

#  DenseNet121 as a feature extractor

Implementing transfer learning to improve the model's ability to extract features and efficient training

In [24]:
# Required Libraries
import numpy as np
import tensorflow as tf
import cv2
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import datetime
from imblearn.over_sampling import ADASYN
from tensorflow.keras.layers import ReLU
from tensorflow.keras.models import Model
from tensorflow.keras.applications import DenseNet121

# Custom decision threshold
OVERROLL_THRESHOLD = 0.7
BROWN_STREAK_THRESHOLD = 0.4

# Setting the seed for reproducibility
np.random.seed(seed)
tf.random.set_seed(seed)

# Load and crop images
folders = ['./Cassava/brown_streak/', './Cassava/healthy1', './Cassava/mosaic_disease1']
original_images, labels = load_images_and_labels(folders)

# Convert cropped images into a format suitable for training
cropped_images_array = np.array([cv2.resize(img, (224, 224)) for img in original_images]) / 255.0

# Encode labels
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)
categorical_labels = to_categorical(encoded_labels)

# Create a StratifiedKFold object
stratified_k_fold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)

# Define class weights
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(labels), y=labels)
class_weight_dict = {i: class_weights[i] for i in range(len(class_weights))}

# Create a data augmentation generator
train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    shear_range=0.2,
    vertical_flip=True,
    horizontal_flip=True,
    fill_mode='nearest')

# Compute confusion matrices for each class
confusion_matrices = []

# Initialize metrics dictionary for precision, recall, F1 score, and accuracy
class_metrics = {'precision': {}, 'recall': {}, 'f1-score': {}, 'accuracy': {}}

# Initialize base model
base_model = DenseNet121(include_top=False, weights='imagenet', input_shape=(224, 224, 3), pooling='avg')

fold_count = 1

# K-Fold Cross-Validation loop
for train_index, val_index in stratified_k_fold.split(cropped_images_array, encoded_labels):
    
    X_train, X_val = cropped_images_array[train_index], cropped_images_array[val_index]
    y_train, y_val = categorical_labels[train_index], categorical_labels[val_index]    
    
    # Apply ADASYN to the training data only
    adasyn = ADASYN()
    X_train_reshaped = X_train.reshape(X_train.shape[0], -1)
    X_train_adasyn, y_train_adasyn = adasyn.fit_resample(X_train_reshaped, y_train)
    X_train_adasyn = X_train_adasyn.reshape(X_train_adasyn.shape[0], 224, 224, 3)
    
    # Create data generator for the training set
    train_generator = train_datagen.flow(X_train_adasyn, y_train_adasyn,batch_size=BATCH_SIZE, seed=seed)
    
    # Freeze the pre-trained layers
    for layer in base_model.layers:
        layer.trainable = False

    # Add new layers on top
    x = base_model.output
    x = Flatten()(x)
    x = Dense(256)(x)
    x = ReLU()(x)  
    x = Dropout(0.5)(x) 
    x = ReLU()(x)  
    x = Dropout(0.3)(x) 
    predictions = Dense(len(label_encoder.classes_), activation='softmax')(x)

    # Compile the model
    model = Model(inputs=base_model.input, outputs=predictions)

    initial_learning_rate = 0.001
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate,
        decay_steps=10000,
        decay_rate=0.9)
    optimizer = Adam(learning_rate=lr_schedule)
    
    # Compile the model
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    
    # Define TensorBoard callback
    log_dir = "logs/fit/Densenet121 and ADASYN" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

    early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
    
    # Fit the model using the data generator
    history = model.fit(train_generator, validation_data=(X_val, y_val), epochs=25, callbacks=[tensorboard_callback,
                                                                                               early_stopping], 
                                                                                               class_weight=class_weight_dict)

    model.save(f"dense121-model_{fold_count}.h5")

    fold_count = fold_count + 1
        
    # Model evaluation and metrics calculation
    y_pred = model.predict(X_val)
    y_true = np.argmax(y_val, axis=1)
    
    # Apply custom decision thresholds
    brown_streak_index = label_encoder.transform(['brown_streak'])[0]  # Get the index for the 'brown_streak' class
    y_pred_binary = np.zeros_like(y_pred)
    y_pred_binary[y_pred > OVERROLL_THRESHOLD] = 1 
    y_pred_binary[(y_pred[:, brown_streak_index] > BROWN_STREAK_THRESHOLD), brown_streak_index] = 1   # Special threshold for brown_streak
    
    # Convert to class labels based on custom decision thresholds
    y_pred_classes = np.argmax(y_pred_binary, axis=1)
            
    # Compute confusion matrix and add to the list
    cnf_matrix = confusion_matrix(y_true, y_pred_classes)
    confusion_matrices.append(cnf_matrix)

    # Calculate and accumulate the metrics
    report = classification_report(y_true, y_pred_classes, output_dict=True, zero_division=1)
    
    for lbl in label_encoder.classes_:
        class_idx = label_encoder.transform([lbl])[0]
        class_metrics['precision'][lbl] = class_metrics['precision'].get(lbl, []) + [report[str(class_idx)]['precision']]
        class_metrics['recall'][lbl] = class_metrics['recall'].get(lbl, []) + [report[str(class_idx)]['recall']]
        class_metrics['f1-score'][lbl] = class_metrics['f1-score'].get(lbl, []) + [report[str(class_idx)]['f1-score']]
        class_metrics['accuracy'][lbl] = class_metrics['accuracy'].get(lbl, []) + [accuracy_score(y_true[y_true==class_idx],
                                                                                                   y_pred_classes[y_true==class_idx])]



Loading images from Cassava\brown_streak
Loading images from Cassava\healthy1
Loading images from Cassava\mosaic_disease1
Loaded labels: ['brown_streak' 'healthy1' 'mosaic_disease1']
Epoch 1/25
26/26 [==============================] - 61s 2s/step - loss: 1.2222 - accuracy: 0.5550 - val_loss: 0.5245 - val_accuracy: 0.7963
Epoch 2/25
26/26 [==============================] - 52s 2s/step - loss: 0.5694 - accuracy: 0.7666 - val_loss: 0.7030 - val_accuracy: 0.6389
Epoch 3/25
26/26 [==============================] - 53s 2s/step - loss: 0.4206 - accuracy: 0.8138 - val_loss: 0.3548 - val_accuracy: 0.8426
Epoch 4/25
26/26 [==============================] - 53s 2s/step - loss: 0.3338 - accuracy: 0.8549 - val_loss: 0.4342 - val_accuracy: 0.7963
Epoch 5/25
26/26 [==============================] - 50s 2s/step - loss: 0.2564 - accuracy: 0.8888 - val_loss: 0.2956 - val_accuracy: 0.8796
Epoch 6/25
26/26 [==============================] - 54s 2s/step - loss: 0.2505 - accuracy: 0.8888 - val_loss: 0.2427 

In [ ]:
# Averaging the metrics
for metric, class_data in class_metrics.items():
    for lbl, values in class_data.items():
        class_metrics[metric][lbl] = np.mean(values)

# Visualizing the metrics
for metric_name in ['f1-score', 'accuracy']:
    plot_metrics(class_metrics[metric_name], metric_name)

# Sample three images from each class/label
sample_images = []
sample_labels = []

for lbl in label_encoder.classes_:
    class_idx = label_encoder.transform([lbl])[0]
    indices = np.where(np.array(y_true) == class_idx)[0][:3]  # Sample three images per class
    sample_images.extend(X_test[indices])
    sample_labels.extend([lbl] * 3)

# Resize sampled images to match the model's input shape
sample_images_resized = [cv2.resize(img, (224, 224)) for img in sample_images]

# Generate predictions for the sampled images with the same thresholds as during validation
sample_predictions = model.predict(np.array(sample_images_resized))

# Apply custom decision thresholds
brown_streak_index = label_encoder.transform(['brown_streak'])[0]  # Get the index for the 'brown_streak' class
sample_predictions_binary = np.zeros_like(sample_predictions)
sample_predictions_binary[sample_predictions > OVERROLL_THRESHOLD] = 1  
sample_predictions_binary[(sample_predictions[:, brown_streak_index] > BROWN_STREAK_THRESHOLD), brown_streak_index] = 1   # Special threshold for brown_streak

# Convert to class labels based on custom decision thresholds
sample_predictions_classes = np.argmax(sample_predictions_binary, axis=1)

# Decode the true and predicted labels to their original string labels
decoded_true_labels = label_encoder.inverse_transform([label_encoder.transform([lbl])[0] for lbl in sample_labels])
decoded_predicted_labels = label_encoder.inverse_transform(sample_predictions_classes)

# Plot the sampled images along with their true and predicted labels
plt.figure(figsize=(15, 15))
for i in range(len(sample_images)):
    plt.subplot(3, 3, i+1)
    plt.imshow(sample_images[i])
    plt.title(f"True: {decoded_true_labels[i]}\nPred: {decoded_predicted_labels[i]}", fontsize=10)
    plt.axis('off')
plt.tight_layout()
plt.show()


The model displayed remarkable accuracy and precision, especially in identifying mosaic disease and brown streak. These findings demonstrate the effectiveness of these strategies in optimizing the use of training data and enhancing the overall model performance for cassava leaf disease classification. With the small and limited image data of brown streak disease, it must be noted that additional data for training may be needed to improve its F1 score. 

# Final Model Evaluation

The model is tested against 107 images from dataverse.harvard.edu https://doi.org/10.7910/DVN/T4RB0B covering the three leaf conditions for model evaluation.

In [ ]:
# Required Libraries
import numpy as np
import tensorflow as tf
import cv2
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import datetime
from imblearn.over_sampling import ADASYN
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Define class names (folder names)
class_names = ['brown_streak', 'healthy1', 'mosaic_disease1']

def plot_metrics(metrics, metric_name):
    # Extract the desired metric for each class
    values = [metrics[label][metric_name] for label in class_names]

    # Calculate the mean of the metric
    mean_metric = np.mean(values)

    # Generate colors from viridis colormap within the range [4/4.5, 4.5/7]
    colormap = cm.viridis(np.linspace(4/4.5, 4.5/7, len(class_names)))

    # Create a bar chart to visualize the metrics
    plt.figure(figsize=(10, 6))
    plt.bar(class_names, values, color=colormap)
    plt.axhline(mean_metric, color='red', linestyle='--', label=f'Mean {metric_name}: {mean_metric:.2f}')
    plt.xlabel('Leaf Condition')
    plt.ylabel(metric_name)
    plt.title(f'{metric_name} by Leaf Condition')
    plt.legend()
    plt.xticks(rotation=45)
    plt.show()

# Custom decision threshold
OVERROLL_THRESHOLD = 0.7
BROWN_STREAK_THRESHOLD = 0.4

# Setting the seed for reproducibility
np.random.seed(seed)
tf.random.set_seed(seed)

# Load and crop images
folders = ['./Cassava Test/brown_streak/', './Cassava Test/healthy1', './Cassava Test/mosaic_disease1']
original_images, labels = load_images_and_labels(folders)

# Convert cropped images into a format suitable for testing
cropped_images_array = np.array([cv2.resize(img, (224, 224)) for img in original_images]) / 255.0

# Encode labels
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)
categorical_labels = to_categorical(encoded_labels)

# Predict on test data
test_predictions = model.predict(cropped_images_array)

# Apply custom decision thresholds to test predictions
test_predictions_binary = np.zeros_like(test_predictions)
test_predictions_binary[test_predictions > OVERROLL_THRESHOLD] = 1
test_predictions_binary[(test_predictions[:, brown_streak_index] > BROWN_STREAK_THRESHOLD), brown_streak_index] = 1

# Convert to class labels based on custom decision thresholds
test_predictions_classes = np.argmax(test_predictions_binary, axis=1)

# Decode the true labels
decoded_test_labels = label_encoder.inverse_transform(encoded_labels)

# Evaluate the model
test_accuracy = accuracy_score(encoded_labels, test_predictions_classes)

# Generate the classification report as a dictionary
test_classification_report = classification_report(encoded_labels, test_predictions_classes, target_names=class_names, output_dict=True)

# Print the classification report
print(test_classification_report)

# Visualize the metrics
plot_metrics(test_classification_report, 'f1-score')

# Calculate accuracy separately
accuracy = accuracy_score(encoded_labels, test_predictions_classes)
print(f"Accuracy: {accuracy:.2f}")

# Sample and visualize predictions
sample_test_indices = np.random.choice(len(original_images), 9, replace=False)

# Resize sampled test images to match the model's input shape
sample_test_images_resized = [cv2.resize(original_images[i], (224, 224)) for i in sample_test_indices]

# Sample predictions based on custom thresholds
sample_test_predictions = test_predictions_binary[sample_test_indices]
sample_test_predictions_classes = np.argmax(sample_test_predictions, axis=1)

# Decode true and predicted labels
sample_true_labels = [decoded_test_labels[i] for i in sample_test_indices]
sample_predicted_labels = label_encoder.inverse_transform(sample_test_predictions_classes)


These results indicate that the model performs exceptionally well classifying healthy leaves with high precision, recall, and F1-score. However, the model's performance is relatively low for brown streak disease, as reflected in lower precision, recall, and F1-score values. The mosaic disease classification falls in between the two. The overall model accuracy is 0.76, indicating a reasonable ability to classify the different leaf conditions.

Improvement areas include using ROC AUC to comprehensively evaluate the model's classification thresholds and possibly attaining more significant samples, especially the underrepresented classes.